测试一下每支股票的最优参数,用图的方式看看最佳参数。

In [106]:
import sys
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import talib
import torch.utils.data as Data

我想要用lstm的网络来预测收盘价。  
x是价格对于，N日前的收盘价的比例
y是收盘价  

# 获取数据

In [107]:
sys.path.append('../../')
import  DataSource
zz500 = DataSource.get_zz500_codes() # 我看中证500的数据
# 我这里查看第一个股票的吧
N = 60 # 多少日的数据
# 我这里仅仅是记录如下的列的涨跌幅度
rate_columns = ['open', 'high', 'low', 'close']
close_index = rate_columns.index('close')
input_size = len(rate_columns)
seq_len = N    # 多少个时间序列的股票
output_size = 1 # 我只是输出一个今天的收盘价相比较昨日的涨跌比率
num_layers = 10 # 多少个gru合并
hidden_size = 20 # 隐藏层的宽度
batch_size = 16 # 一个批次有多少个数据
lr = 0.0001

In [108]:
def get_loader(code_name, N=60, rate_columns=['open', 'high', 'low', 'close'], batch_size=16, train_rate=0.9):
    dt = DataSource.get_data(code_name)
    dt_list = []
    for j in range(N+1):
        # N+1表示要多一天，比如前面30天，我要看看第31天的收盘价相对于第一天的收盘价是什么比例
        for i in rate_columns:
            dt_tmp = pd.DataFrame({f'{i}_rate_{j}':(dt[f'{i}'].shift(-j) - dt[f'preclose'])/dt[f'preclose']})
            dt_list.append(dt_tmp)
    
    dt2 = pd.concat(dt_list[:-(len(rate_columns))], axis=1)
    # x
    x_price = dt2.to_numpy()
    x_price = x_price[:-(N+1)]
    # 更改维度
    x_shape_old = x_price.shape
    x_shape_new_1 = x_shape_old[0]
    x_shape_new_3 = len(rate_columns)
    x_shape_new_2 = int(x_shape_old[1] / x_shape_new_3)
    x_price = x_price.reshape((x_shape_new_1, x_shape_new_2, x_shape_new_3))
    # y
    y_close = dt_list[-(len(rate_columns)- rate_columns.index('close'))][f'close_rate_{N}'].to_numpy()[:-(N+1)]
    # 然后是批次的整数倍
    _x_y_len = int(x_price.shape[0]/batch_size) * batch_size
    
    x_price = x_price[-_x_y_len:]
    y_close = y_close[-_x_y_len:]
    x_price = x_price.reshape(x_price.shape[0]//batch_size, batch_size, x_price.shape[1], x_price.shape[2])
    y_close = y_close.reshape(y_close.shape[0]//batch_size, batch_size, 1)
    # 做成加载器
    dataset = Data.TensorDataset(torch.FloatTensor(x_price), torch.FloatTensor(y_close))
    train_loader, test_loader = Data.random_split(dataset,
                                                 lengths=[int(train_rate*len(dataset)), len(dataset)- int(train_rate*len(dataset))],
                                                 generator=torch.Generator().manual_seed(0))
    return train_loader, test_loader



In [109]:

# 这里做一个网络
import torch.nn as nn
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        out, _ = self.gru(x) # gru层
        out = self.fc(out[:,-1, :]) # 全连接层
        return out

In [110]:
# 训练
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = GRUModel(input_size, hidden_size, num_layers, output_size).to(device)
criterion = torch.nn.MSELoss() # 均方差误差函数
optimizer = optim.Adam(net.parameters(), lr=lr)

In [111]:
# 这里要试试绘图
# 这里要试试绘图
from visdom import Visdom
import torch
import numpy as np
from PIL import Image

In [112]:
 # 获取绘图对象，相当于plt
 vis = Visdom()

Setting up a new session...


In [113]:
# 创建窗口并初始化
vis.line([0.], [0], win='train_loss', opts=dict(title='train_loss'))
train_loader, test_loader = get_loader(zz500[0],batch_size=batch_size, train_rate=0.9)
# 我这里用测试集来测试
_step = 1
for epoch in range(10):  # 迭代
    running_loss_2 = []
    for i, data in enumerate(train_loader, 0):
            # 获取输入
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            # 正向传播，反向传播，优化
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # 这里更新到显示
            vis.line([loss.cpu().detach().numpy()], [_step], win='train_loss', update='append')
            _step = _step + 1
print(f'train last loss : {loss.item()}')
# 接下来做测试
_step = 1
vis.line([0.], [0], win='test_loss', opts=dict(title='test_loss'))
with torch.no_grad():
    for i, data in enumerate(test_loader, 0):
            # 获取输入
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            # 正向传播，反向传播，优化
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            # 这里更新到显示
            vis.line([loss.cpu().detach().numpy()], [_step], win='test_loss', update='append')
            _step = _step + 1
print(f'test last loss : {loss.item()}')
print('finish')

train last loss : 0.0001622647832846269
test last loss : 0.0035867190454155207
finish


[WinError 10054] 远程主机强迫关闭了一个现有的连接。
[WinError 10054] 远程主机强迫关闭了一个现有的连接。
[WinError 10054] 远程主机强迫关闭了一个现有的连接。
[WinError 10054] 远程主机强迫关闭了一个现有的连接。
Visdom.setup_socket.<locals>.on_close() takes 1 positional argument but 3 were given
[WinError 10054] 远程主机强迫关闭了一个现有的连接。
[WinError 10054] 远程主机强迫关闭了一个现有的连接。
Visdom.setup_socket.<locals>.on_close() takes 1 positional argument but 3 were given
[WinError 10054] 远程主机强迫关闭了一个现有的连接。
[WinError 10054] 远程主机强迫关闭了一个现有的连接。
[WinError 10054] 远程主机强迫关闭了一个现有的连接。
Visdom.setup_socket.<locals>.on_close() takes 1 positional argument but 3 were given
[WinError 10054] 远程主机强迫关闭了一个现有的连接。
[WinError 10054] 远程主机强迫关闭了一个现有的连接。
Visdom.setup_socket.<locals>.on_close() takes 1 positional argument but 3 were given
Visdom.setup_socket.<locals>.on_close() takes 1 positional argument but 3 were given
Visdom.setup_socket.<locals>.on_close() takes 1 positional argument but 3 were given
[WinError 10054] 远程主机强迫关闭了一个现有的连接。
Visdom.setup_socket.<locals>.on_close() takes 1 positional argument bu